In [ ]:
%matplotlib inline
import pandas as pd
import warnings
import numpy as np
from numpy import NaN
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [ ]:
warnings.simplefilter("ignore")

In [ ]:
url = "https://raw.githubusercontent.com/AndresBeta123/ProyectoDataMining-2023/main/title_complete_preprocesado.csv"
title_complete_preprocesado = pd.read_csv(url)

In [ ]:
url = "https://raw.githubusercontent.com/AndresBeta123/ProyectoDataMining-2023/main/principals_complete_preprocesado.csv"
principals_complete_preprocesado = pd.read_csv(url)

# Asociacion

In [ ]:
def generarMatrizDeAsociacion(data):
    all_items = set()
    for items_str in data:
        items_list = [item.strip() for item in items_str.split(',')]
        all_items.update(items_list)

    # Crear un diccionario para almacenar los datos
    data_dict = {}
    for item in all_items:
        data_dict[item] = []

    # Llenar el diccionario con los valores True/False
    for items_str in data:
        #data_dict['ID'].append(tid)
        items_list = [item.strip() for item in items_str.split(',')]
        for item in all_items:
            data_dict[item].append(item in items_list)

    # Crear un DataFrame a partir del diccionario
    return pd.DataFrame(data_dict)

In [ ]:
def ReglasDeAsociacionApriori(df,support,confidence):
    frequent_itemsets = apriori(df, min_support=support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric='lift')
    rules = rules[rules['confidence'] >= confidence]
    rules.sort_values(by='confidence')
    display(rules)

In [ ]:
def ReglasDeAsociacionfpgrowth(df,support,confidence):
    frequent_itemsets = fpgrowth(df, min_support=support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric='lift')
    rules = rules[rules['confidence'] >= confidence]
    rules.sort_values(by='confidence')
    display(rules)

In [ ]:
data_generes_notNaN = title_complete_preprocesado['genres'].dropna()
data_generes_NaN = title_complete_preprocesado['genres'].fillna('sinGenero')

Matrix_generes_notNaN = generarMatrizDeAsociacion(data_generes_notNaN)
Matrix_generes_NaN = generarMatrizDeAsociacion(data_generes_NaN)

In [ ]:
minSupport = 0.0001
minConfidence = 0.5

## Apriori

In [ ]:
ReglasDeAsociacionApriori(Matrix_generes_notNaN,minSupport,minConfidence)

In [ ]:
ReglasDeAsociacionApriori(Matrix_generes_NaN,minSupport,minConfidence)

## FP-Growth

In [ ]:
ReglasDeAsociacionfpgrowth(Matrix_generes_notNaN,minSupport,minConfidence)

In [ ]:
ReglasDeAsociacionfpgrowth(Matrix_generes_NaN,minSupport,minConfidence)

# Clustering

In [ ]:
import sklearn.cluster as sk

In [ ]:
def plot_Codo(data,k):
    K = range(2,k)
    Sum_of_squared_distances = []
    data = data.dropna()
    for i in K:
        data_clustered =  sk.KMeans(n_clusters=i, random_state=0,verbose = 0).fit(data)
        Sum_of_squared_distances.append(data_clustered.inertia_)
    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('SSE')
    plt.show()

In [ ]:
title_complete_preprocesado

In [ ]:
dimensiones = ['runtimeMinutes','averageRating','numVotes']
data_for_clustering = title_complete_preprocesado[dimensiones].copy()
plot_Codo(data_for_clustering,10)

In [ ]:
dimensiones = ['averageRating','numVotes']
data_for_clustering = title_complete_preprocesado[dimensiones].copy()
plot_Codo(data_for_clustering,10)

In [ ]:
dimensiones = ['averageRating']
data_for_clustering = title_complete_preprocesado[dimensiones].copy()
plot_Codo(data_for_clustering,10)

In [ ]:
dimensiones = ['category 1','averageRating']
data_for_clustering = title_complete_preprocesado[dimensiones].copy()
plot_Codo(data_for_clustering,10)

In [ ]:
dimensiones = ['category 1']
data_for_clustering = title_complete_preprocesado[dimensiones].copy()
plot_Codo(data_for_clustering,10)

In [ ]:
data_clustered =  sk.KMeans(n_clusters=3, random_state=0,verbose = 0).fit(data_for_clustering)
all_preds = data_clustered.inertia_
all_clusters = data_clustered.cluster_centers_
display(all_preds)
display(all_clusters)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(np.array(data_for_clustering)[:,0],np.array(data_for_clustering)[:,1],c='b')
plt.scatter(all_clusters[:,0],all_clusters[:,1],c="r")